## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-03-02-14-48 +0000,nyt,House Oversight Panel Releases Some Epstein Fi...,https://www.nytimes.com/2025/09/02/us/politics...
1,2025-09-03-02-13-00 +0000,wsj,Australia’s Growth Remained Sluggish as Invest...,https://www.wsj.com/economy/australian-growth-...
2,2025-09-03-02-09-54 +0000,wapo,"China’s Xi, flanked by Putin and Kim, shows of...",https://www.washingtonpost.com/world/2025/09/0...
3,2025-09-03-01-56-19 +0000,bbc,"Putin, Kim and Xi are meeting in China: BBC co...",https://www.bbc.com/news/videos/cwypyq42e13o?a...
4,2025-09-03-01-52-16 +0000,nyt,Xi Parades Firepower to Signal That China Won’...,https://www.nytimes.com/2025/09/02/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
123,trump,48
21,new,14
14,china,13
0,house,13
124,he,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,102
219,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,99
147,2025-09-02-19-04-50 +0000,nypost,‘Morning Joe’ host Joe Scarborough urges defia...,https://nypost.com/2025/09/02/us-news/joe-scar...,92
41,2025-09-02-23-36-33 +0000,cbc,U.S. military kills 11 in strike on alleged dr...,https://www.cbc.ca/news/world/trump-venezuela-...,89
96,2025-09-02-21-04-00 +0000,wsj,Trump’s Use of Troops in Los Angeles Was Unlaw...,https://www.wsj.com/us-news/law/trumps-troop-d...,88


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,102,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...
2,60,2025-09-03-02-09-54 +0000,wapo,"China’s Xi, flanked by Putin and Kim, shows of...",https://www.washingtonpost.com/world/2025/09/0...
41,41,2025-09-02-23-36-33 +0000,cbc,U.S. military kills 11 in strike on alleged dr...,https://www.cbc.ca/news/world/trump-venezuela-...
315,39,2025-09-02-03-43-25 +0000,nypost,Chef killed in electric buggy crash after late...,https://nypost.com/2025/09/01/world-news/chef-...
83,39,2025-09-02-21-29-07 +0000,nypost,Would-be assassin Ryan Wesley Routh challenges...,https://nypost.com/2025/09/02/us-news/would-be...
47,38,2025-09-02-23-13-56 +0000,nypost,Jeffrey Epstein’s victims revealed ‘additional...,https://nypost.com/2025/09/02/us-news/jeffrey-...
36,35,2025-09-03-00-03-52 +0000,latimes,Cardi B wins civil assault trial brought by se...,https://www.latimes.com/entertainment-arts/sto...
61,33,2025-09-02-22-21-30 +0000,nypost,"Zohran Mamdani’s brain trust is full of young,...",https://nypost.com/2025/09/02/us-news/zohran-m...
191,33,2025-09-02-16-46-17 +0000,bbc,UK borrowing costs hit 27-year high adding to ...,https://www.bbc.com/news/articles/cy989njnq2wo...
137,33,2025-09-02-19-58-20 +0000,nypost,EU antitrust regulators delay Google adtech fi...,https://nypost.com/2025/09/02/business/eu-anti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
